In [10]:
!wget https://raw.githubusercontent.com/ivpcl/AOLME_Level2/main/ColorDetectionInterface/AOLMERobots.py
!wget https://raw.githubusercontent.com/ivpcl/AOLME_Level2/main/ColorDetectionInterface/color_threshold_sliders_image.py
!wget https://raw.githubusercontent.com/ivpcl/AOLME_Level2/main/ColorDetectionInterface/thresholds2.py
!wget https://raw.githubusercontent.com/ivpcl/AOLME_Level2/main/ColorDetectionInterface/Bob.jpg


--2022-04-10 22:32:47--  https://raw.githubusercontent.com/ivpcl/AOLME_Level2/main/ColorDetectionInterface/AOLMERobots.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1081 (1.1K) [text/plain]
Saving to: ‘AOLMERobots.py.1’

AOLMERobots.py.1    100%[===================>]   1.06K  --.-KB/s    in 0s      

2022-04-10 22:32:47 (69.3 MB/s) - ‘AOLMERobots.py.1’ saved [1081/1081]

--2022-04-10 22:32:47--  https://raw.githubusercontent.com/ivpcl/AOLME_Level2/main/ColorDetectionInterface/color_threshold_sliders_image.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, await

Bear: [69, 167] [13, 122] [16, 78]

Clothes: [0, 113] [60, 255] [83, 255]

Skin: [115, 255] [150, 255] [0, 135]

Gloves and shoes: [15, 55] [10, 89] [3, 41]

In [11]:
#@title Color threshold{ run: "auto" }
%matplotlib inline
import cv2
import AOLMERobots as gopi
from thresholds2 import *
from ipywidgets import interactive
import ipywidgets as widgets
import matplotlib.pyplot as plt
 
 
image = cv2.imread('Bob.jpg')
height, width = image.shape[:2]
image= cv2.resize(image, (int(320/height*width), 320))
 
thr_obj = threshold(image)
 
 
 
def DisplayData(redLow, redHigh, greenLow, greenHigh, blueLow, blueHigh):
 
  img = image.copy()
 
  # Red component processing:
  thr_obj.sel_color_comp ('red')
  RedRange =  thr_obj.ThreshRange (redLow , redHigh)   
  #RedLow   =  thr_obj.ThreshLow   (redLow)            
  #RedHi    =  thr_obj.ThreshHigh  (redHi)


  # Green component processing:
  thr_obj.sel_color_comp ('green')
  GreenRange =  thr_obj.ThreshRange (greenLow , greenHigh)  
  #GreenLow   =  thr_obj.ThreshLow   (greenLow)            
  #GreenHi    =  thr_obj.ThreshHigh  (greenHi)


  # Blue component processing:
  thr_obj.sel_color_comp ('blue')
  BlueRange =  thr_obj.ThreshRange (blueLow , blueHigh)  
  #BlueLow   =  thr_obj.ThreshLow   (blueLow)            
  #BlueHi    =  thr_obj.ThreshHigh  (blueHi)
  
  # Get middle-RGB combination
  comb_image, comb_allTh= thr_combination(img, BlueRange,GreenRange, RedRange).return_result()   
  area = sum(comb_allTh)


  skip = 70
  # Generate the second window, including 9 single color images, one combination image, and text
  white = np.zeros((image.shape[0], skip), np.uint8)
  white[:] = (255)
  white_3_channel = cv2.merge((white, white, white))
 
  comb_allTh_3_channel = cv2.merge((comb_allTh*255, comb_allTh*255, comb_allTh*255)) 
  row_1 = np.hstack((img, white_3_channel))
  row_2 = np.hstack((row_1, comb_allTh_3_channel))
  row_3 = np.hstack((row_2, white_3_channel))
  row_4 = np.hstack((row_3, comb_image))
  


  white_title = np.zeros((50, row_4.shape[1]), np.uint8)
  white_title[:] = (255)
  white_title_3_channel = cv2.merge((white_title, white_title, white_title))
  
  font = cv2.FONT_HERSHEY_SIMPLEX
  cv2.putText(white_title_3_channel, 'Input', (120,40), font, 0.7, (0,0,0), 2, 0)
  cv2.putText(white_title_3_channel, 'Binary Image', (120+image.shape[0]+skip,40), font, 0.7, (0,0,0), 2, 0)
  cv2.putText(white_title_3_channel, 'Output', (180+image.shape[0]*2+skip*2,40), font, 0.7, (0,0,0), 2, 0)
  all = np.vstack((white_title_3_channel, row_4))

  cv2_imshow(all)


  rgb_values = [blueLow, blueHigh, greenLow, greenHigh, redLow, redHigh]
  
  # Show histograms for RGB color
  gopi.show_RGB_hist(img, rgb_values)
  print('The detected color region has {} pixels'.format(int(area)))

  #print(redLow,'<= Red   <= ', redHigh)
  #print(greenLow,'<= Green <= ', greenHigh)
  #print(blueLow,'<= Blue  <= ', blueHigh)


 
 
interactive_plot = interactive(DisplayData,
                               redLow   = widgets.IntSlider(min=0, max=255, step=1, value=16),
                               redHigh  = widgets.IntSlider(min=0, max=255, step=1, value=156),
                               greenLow = widgets.IntSlider(min=0, max=255, step=1, value=30),
                               greenHigh= widgets.IntSlider(min=0, max=255, step=1, value=132),
                               blueLow  = widgets.IntSlider(min=0, max=255, step=1, value=24),
                               blueHigh = widgets.IntSlider(min=0, max=255, step=1, value=180))
interactive_plot


interactive(children=(IntSlider(value=16, description='redLow', max=255), IntSlider(value=156, description='re…